In [13]:
# This version uses abstracted functions that actually work to load X and Y training data into arrays of the right shape
# data successfully flows through LSTM connected to a flatten and dense layer
# this version is trained on all of part 1 data and the model is saved in a local file

# Here I will be building out the architecture of the first classification LSTM
# At each time step, this LSTM will take in a vector representing the extracted audio and visual features from Liris-ACCEDE
# Its goal is to output whether or not the movie induces fear at each time step


# First, import necessary libraries
import tensorflow as tf
import numpy as np
import keras.backend as K

In [14]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import LSTM, Conv1D
# my custom data_utils file
from data_utils_local08 import *

In [15]:
# uploading the X values
X_input = load_Xinput(get_fc6_directory(7))
print(X_input.shape)

(212, 4096)


In [16]:
# uploading the Y values
y_data_input = fear_oneHot(212, 'fear_annotations_part01/MEDIAEVAL18_7_Fear.txt')
print(y_data_input.shape)

(212,)


In [17]:
# Uploading part01 of the local fc6 data

timesteps = 212   # the number of seconds in movie 07 --> i will figure out how to programmatically get this value
data_dim = 4096    # the number of output values from VGG16 layer fc6 --> switch to programmatic later
num_movies = 4
batch_size = 7
num_epochs = 5
validation_num = 3

# set up the X_train_data master array
X_train_data = np.zeros([num_movies, timesteps, data_dim]) #oooooh this array will have to be as long as the longest
                                                            # movie and padded with zeros --> this won't cause problems
                                                            # right?
X_valid_data = np.zeros([validation_num, timesteps, data_dim])
        
# for each movie number between and including 7 and 13
for num in range(num_movies):
    # load the X_input data
    X_input = load_Xinput(get_fc6_directory(7 + num))
    # put this X_input data into the X_train_data array
    X_train_data[num, :, :] = X_input
print(X_train_data.shape)



(4, 212, 4096)


In [18]:
# loading X validation set
X_valid_data = np.zeros([validation_num, timesteps, data_dim])

for num in range(validation_num):
    # load the X_input data
    X_valid = load_Xinput(get_fc6_directory(7 + num_movies + num))
    # put this X_input data into the X_train_data array
    X_valid_data[num, :, :] = X_valid
print(X_valid_data.shape)


(3, 212, 4096)


In [19]:
# uploading the y data

# set up y_train_data master array
Y_train_data = np.zeros([num_movies, timesteps])

# for each movie number between and including 7 and 13
for num in range(num_movies):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+num))
    # create a one-hot vector
    y_data = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_train_data[num, :] = y_data
print(Y_train_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [20]:
# upload the Y validation set
Y_valid_data = np.zeros([validation_num, timesteps])

# for each movie number in validation set
for num in range(validation_num):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+ num_movies + num))
    # create a one-hot vector
    y_valid = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_valid_data[num, :] = y_valid
print(Y_valid_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [21]:
# constructing a many-to-one LSTM model in keras --> inspiration: https://stackoverflow.com/questions/43034960/many-to-one-and-many-to-many-lstm-examples-in-keras
# i will start by training a model on only the VGG16 fc6 layer output (that's just one feature)
# should I eventually abstract this LSTM model? Create its own object file?
model = Sequential()

# add a convolutional layer over here
model.add(Conv1D(10, kernel_size=3, activation="relu", input_shape=(timesteps, data_dim)))

model.add(LSTM(212, return_sequences=True))

# going to try adding a flatten layer in here
model.add(Flatten()) # I got this from a github thing, but I still don't completely understand why it works

# add the final dense layer and then softmax
model.add(Dense(212, activation='sigmoid'))
# going to add a softmax activation to this
#model.add(Activation('softmax'))

In [22]:
# a new metric of evaluation! the F-score!
def FScore2(y_true, y_pred):
    '''
    The F score, beta=2
    '''
    B2 = K.variable(4)
    OnePlusB2 = K.variable(5)
    pred = K.round(y_pred)
    tp = K.sum(K.cast(K.less(K.abs(pred - K.clip(y_true, .5, 1.)), 0.01), 'float32'), -1)
    fp = K.sum(K.cast(K.greater(pred - y_true, 0.1), 'float32'), -1)
    fn = K.sum(K.cast(K.less(pred - y_true, -0.1), 'float32'), -1)

    f2 = OnePlusB2 * tp / (OnePlusB2 * tp + B2 * fn + fp)

    return K.mean(f2)

In [23]:
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy', FScore2])

In [27]:
# running the LSTM model
model.fit(X_train_data, Y_train_data, epochs = 50, validation_data=(X_valid_data, Y_valid_data))
print("finished training!")

Train on 4 samples, validate on 3 samples
Epoch 1/50
4/4 [==============================] - 1s 141ms/step - loss: 0.0046 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 1.9212 - val_binary_accuracy: 0.7028 - val_FScore2: 0.0768
Epoch 2/50
4/4 [==============================] - 1s 212ms/step - loss: 0.0042 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 1.9331 - val_binary_accuracy: 0.7028 - val_FScore2: 0.0768
Epoch 3/50
4/4 [==============================] - 1s 132ms/step - loss: 0.0039 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 1.9444 - val_binary_accuracy: 0.7028 - val_FScore2: 0.0768
Epoch 4/50
4/4 [==============================] - 1s 175ms/step - loss: 0.0036 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 1.9552 - val_binary_accuracy: 0.7028 - val_FScore2: 0.0768
Epoch 5/50
4/4 [==============================] - 0s 120ms/step - loss: 0.0033 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 1.9662 - val_binary_accuracy: 0.7044 - val_FSco

In [25]:
# trying to view the model output
out = model.predict(X_train_data)
print("model prediction:")
print(out[0])
print("target:")
print(Y_train_data[0])

print("before 64:")
print(out[0][63])
print("64:")
print(out[0][65])

print("rounded")
print(np.round(out)[0])

model prediction:
[1.58757117e-04 5.65342561e-05 6.93871916e-05 9.36704746e-05
 2.05786091e-05 1.35061142e-04 1.46761318e-04 8.88979703e-05
 8.49307398e-05 4.90488601e-05 7.02822072e-05 5.13327359e-05
 1.21120866e-04 4.03712647e-06 4.37390408e-05 9.73372735e-05
 9.14012999e-05 1.93071930e-04 6.93227703e-05 9.73926071e-05
 6.34244352e-05 1.31032415e-04 1.38340198e-04 1.01213773e-04
 8.40352950e-05 7.51984262e-05 4.44619945e-04 9.96925402e-04
 4.24151542e-04 4.02316859e-04 4.37093113e-05 8.88102732e-05
 3.79306119e-04 2.04113615e-03 2.29448087e-05 3.79092642e-04
 3.23549611e-05 1.28209433e-02 1.24152517e-02 7.24724552e-04
 2.31764209e-03 6.69773202e-04 7.33688939e-04 8.49537464e-05
 5.54204627e-04 5.04970434e-04 5.53619640e-04 7.17733870e-04
 4.69983643e-04 3.64973275e-05 7.36893271e-04 1.41977950e-03
 1.01963375e-02 8.13827291e-03 3.08922795e-03 7.45249121e-03
 1.05979629e-02 1.30151231e-02 9.43313446e-03 7.21332477e-03
 5.23011375e-04 7.03895639e-04 4.68040002e-04 6.43996405e-04
 9.752

In [26]:
#try visualizing this model at some point?